In [1]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer, execute

## VQE Example with Single Qubit Variational From
Example from: https://qiskit.org/textbook/ch-applications/vqe-molecules.html

"We will now use the simple single qubit variational form to solve a problem similar to ground state energy estimation. Specifically, we are given a random probability vector  $\overrightarrow{x}$ and wish to determine a possible parameterization for our single qubit variational form such that it outputs a probability distribution that is close to $\overrightarrow{x}$ (where closeness is defined in terms of the Manhattan distance between the two probability vectors)."

### Create Target Distribution
This is the distribution we are aiming to simulate. We will use it as a comparison for when we calculate the loss.

In [9]:
#np.random.seed(999999)
target_distr = np.zeros(2)
#target_distr = np.random.rand(2)
#target_distr /= sum(target_distr)
target_distr[0] = 0.8
target_distr[1] = 0.2
target_distr

array([0.8, 0.2])

### Set up Variational Form
Create a quantum circuit with one qubit an one measurment register. Apply the U3 gate with some arbitrary set of params to the qubit. The params are fixed but were pulled from a Gaussian distribution. 

In [12]:
#params = np.random.rand(3)
params = np.zeros(3)
params[0] = 0.71981
params[1] = 0.081255
params[2] = 0.653

##
## set up variational form
##
qr = QuantumRegister(1, name="q")
cr = ClassicalRegister(1, name='c')
qc = QuantumCircuit(qr, cr)
qc.u3(params[0], params[1], params[2], qr[0])
qc.measure(qr, cr[0])

In [13]:
print(qc)

        ┌────────────────────────────┐┌─┐
q_0: |0>┤ U3(0.71981,0.081255,0.653) ├┤M├
        └────────────────────────────┘└╥┘
 c_0: 0 ═══════════════════════════════╩═
                                         


### Perform Simulation of Variational Form
Measure the circuits state some number of times set by `num_shots`.

In [23]:
##
## perform simulation
##
backend = Aer.get_backend("qasm_simulator")
num_shots = 1000
result = execute(qc, backend, shots=num_shots).result()
counts = result.get_counts(qc)
output_distr = [v/num_shots for v in counts.values()]
cost = sum([np.abs(output_distr[i] - target_distr[i]) for i in range(2)])
print(output_distr)
print(cost)

[0.112, 0.888]
1.376


### Optimize

In [10]:
from qiskit.aqua.components.optimizers import COBYLA

# Initialize the COBYLA optimizer
optimizer = COBYLA(maxiter=500, tol=0.0001)

In [24]:
# Create the initial parameters (noting that our single qubit variational form has 3 parameters)
#params = np.random.rand(3)
ret = optimizer.optimize(num_vars=3, objective_function=cost, initial_point=params)

TypeError: 'numpy.float64' object is not callable

In [ ]:

# Obtain the output distribution using the final parameters
qc = get_var_form(ret[0])
counts = execute(qc, backend, shots=NUM_SHOTS).result().get_counts(qc)
output_distr = get_probability_distribution(counts)

print("Target Distribution:", target_distr)
print("Obtained Distribution:", output_distr)
print("Output Error (Manhattan Distance):", ret[1])
print("Parameters Found:", ret[0])